*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv(train_file_path, sep='\t', names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', names=['label', 'message'])
train_data.tail()

In [ ]:
train_data.describe()

In [ ]:
ham_msg = train_data[train_data.label == 'ham']
spam_msg = train_data[train_data.label == 'spam']

ham_msg_text = ' '.join(ham_msg.message.to_numpy().tolist())
spam_msg_text = ' '.join(spam_msg.message.to_numpy().tolist())

In [ ]:
ham_msg_cloud = WordCloud(
    width=520, 
    height=260, 
    stopwords=STOPWORDS,
    max_font_size=50,
    background_color='black',
    colormap='Blues'
).generate(ham_msg_text)

plt.figure(figsize=(16,10))
plt.imshow(ham_msg_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
spam_msg_cloud = WordCloud(
    width=520, 
    height=260, 
    stopwords=STOPWORDS,
    max_font_size=50,
    background_color='black',
    colormap='Blues'
).generate(spam_msg_text)

plt.figure(figsize=(16,10))
plt.imshow(spam_msg_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
ham_msg_df = ham_msg.sample(n = len(spam_msg), random_state=44)
spam_msg_df = spam_msg

msg_df = ham_msg_df.append(spam_msg_df).reset_index(drop=True)
plt.figure(figsize=(8,6))

sns.countplot(msg_df.label)

In [ ]:
msg_df['text_length'] = msg_df['message'].apply(len)

labels = msg_df.groupby('label').mean()
labels

In [ ]:
msg_df['msg_type'] = msg_df['label'].map({'ham': 0, 'spam': 1})
msg_label = msg_df['msg_type'].values

train_features, test_features, train_labels, test_labels = train_test_split(
    msg_df['message'], 
    msg_label, 
    test_size=0.2, 
    random_state=434
)

In [ ]:
# Hyperparams
max_len = 50
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
vocab_size = 500

tokenizer = Tokenizer(
    num_words=vocab_size, 
    char_level=False, 
    oov_token = oov_tok
)
tokenizer.fit_on_texts(train_features)

In [ ]:
word_index = tokenizer.word_index
word_index

In [ ]:
training_sequences = tokenizer.texts_to_sequences(train_features)
training_padded = pad_sequences(
    training_sequences, 
    maxlen = max_len, 
    padding = padding_type, 
    truncating = trunc_type
)

testing_sequences = tokenizer.texts_to_sequences(test_features)
testing_padded = pad_sequences(
    testing_sequences,
    maxlen = max_len,
    padding = padding_type,
    truncating = trunc_type
)

In [ ]:
training_padded[0]

In [ ]:
# Hyperparams
vocab_size = 500
embedding_dim = 16
drop_value = 0.2
n_dense = 24

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(24, activation='relu'))
model.add(Dropout(drop_value))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(
    training_padded,
    train_labels,
    epochs=num_epochs,
    validation_data=(testing_padded, test_labels),
    callbacks=[early_stop],
    verbose=2
)

In [ ]:
model.evaluate(testing_padded, test_labels)

In [ ]:
metrics = pd.DataFrame(history.history)
metrics.rename(
    columns={'loss': 'Training_Loss', 
             'accuracy': 'Training_Accuracy',
             'val_loss': 'Validation_Loss',
             'val_accuracy': 'Validation_Accuracy'},
    inplace=True
)

def plot_graphs1(var1, var2, string):
  metrics[[var1, var2]].plot()
  plt.title('Training and Validation ' + string)
  plt.xlabel('Number of epochs')
  plt.ylabel(string)
  plt.legend([var1, var2])

In [ ]:
plot_graphs1('Training_Loss', 'Validation_Loss', 'loss')  

In [ ]:
plot_graphs1('Training_Accuracy', 'Validation_Accuracy', 'accuracy')

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  new_seq = tokenizer.texts_to_sequences([pred_text])
  padded = pad_sequences(
      new_seq, 
      maxlen=max_len,
      padding=padding_type,
      truncating=trunc_type
  )

  prediction = model.predict(padded)
  pred = [prediction[0][0]]

  if prediction[0][0] < 0.1:
    pred.append('ham')
  else:
    pred.append('spam')

  return pred

pred_text = "Would you like to try our product? Contact us now!"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
